In [39]:
import mysql.connector

# #Connect to DB 
conn=mysql.connector.connect(host="localhost",user="root",password="Cuppuchinno!1",database="testdb")

#Established the connection
cursor= conn.cursor()

print("DB Connected Successfully")

DB Connected Successfully


In [40]:
#Import libraries 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns

In [41]:
pip install pandas openpyxl

In [42]:
#Read the excel file (Dataset)

import os

downloads_folder = os.path.expanduser('~/Downloads')

# Specify the name of your Excel file
excel_file_name = 'training_retail_supply_chain_data_test.xlsx' 

# Construct the full path to the Excel file
file_path = os.path.join(downloads_folder, excel_file_name)

try:
    # Read the Excel file into a pandas DataFrame
    # You can specify a sheet name if your file has multiple sheets
    # For example: df = pd.read_excel(file_path, sheet_name='Sheet1')
    df = pd.read_excel(file_path)

    # Display the first few rows of the DataFrame
    print("Successfully read the Excel file:")
  

except FileNotFoundError:
    print(f"Error: The file '{excel_file_name}' was not found in your Downloads folder.")

df.head()

Successfully read the Excel file:


Date  Product_ID Product_Name     Category  Units_Sold  Revenue  \
0 2023-01-01         199    Product E         Home          24     3930   
1 2023-01-02         154    Product E         Toys          46     4153   
2 2023-01-03         190    Product C  Electronics          19     2815   
3 2023-01-04         192    Product C  Electronics          30     4873   
4 2023-01-05         137    Product B        Books          30     4981   

   Stock_Level  Reorder_Threshold  Supplier_ID  Supplier_Rating  \
0           15                 18          287                3   
1          138                 32          291                7   
2          259                 16          263                9   
3           71                 24          242                8   
4           99                 41          228                2   

   Discount_Applied  Return_Flag  
0                20            0  
1                10            1  
2                 0            1  
3                 0            0  
4                 0            1

In [43]:
def get_mysql_type(pandas_dtype):
    if 'int' in str(pandas_dtype):
        return 'INT'
    elif 'float' in str(pandas_dtype):
        return 'FLOAT'
    elif 'bool' in str(pandas_dtype):
        return 'BOOLEAN'
    elif 'datetime' in str(pandas_dtype):
        return 'DATETIME'
    else: # Default to VARCHAR for strings and objects
        return 'VARCHAR(255)'

def generate_create_table_sql(df, table_name):
    columns_sql = []
    for col_name, dtype in df.dtypes.items():
        mysql_type = get_mysql_type(dtype)
        columns_sql.append(f"`{col_name}` {mysql_type}")
        
    create_table_sql = f"CREATE TABLE IF NOT EXISTS `{table_name}` ({', '.join(columns_sql)})"
    return create_table_sql

In [44]:
# Create a table in mysql
table_name = 'Retail_Data_New_test'
create_table_sql = generate_create_table_sql(df, table_name)
cursor.execute(create_table_sql)
print(f"Table '{table_name}' created successfully (or already exists).")


Table 'Retail_Data_New_test' created successfully (or already exists).


In [45]:
# insert data into Retail_Data_New_test 
# Table name in your MySQL database
table_name = 'Retail_Data_New_test'

try:
    # Establish a connection to the MySQL database
    mydb = conn
    mycursor = mydb.cursor()

    # Construct the INSERT statement
    # Ensure the column names in the DataFrame match the table columns
    columns = ', '.join(df.columns)
    placeholders = ', '.join(['%s'] * len(df.columns))
    sql = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

    # Prepare data for insertion
    # Convert DataFrame rows to a list of tuples
    records_to_insert = [tuple(row) for row in df.values]

    # Execute the INSERT statement for multiple rows
    mycursor.executemany(sql, records_to_insert)

    # Commit the changes to the database
    mydb.commit()

    print(f"{mycursor.rowcount} records inserted successfully into '{table_name}'.")
    

except mysql.connector.Error as err:
    print(f"Error: {err}")



10000 records inserted successfully into 'Retail_Data_New_test'.


In [46]:
from sqlalchemy import create_engine
import pandas as pd
import os

SQL_QUERY = "SELECT * FROM Retail_Data_New_test;"

excel_file_path = os.path.join(downloads_folder, 'Output_file.xlsx')
df = pd.read_sql(SQL_QUERY, conn)
print(f"Retrieved {len(df)} rows from the database.")

        # Ensure the 'downloads' folder exists
os.makedirs(downloads_folder, exist_ok=True)

        # Export DataFrame to Excel
df.to_excel(excel_file_path, index=False)
print(f"Data successfully exported to {excel_file_path}")

C:\Users\Hp\AppData\Local\Temp\ipykernel_8884\768704565.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(SQL_QUERY, conn)


Retrieved 10000 rows from the database.
Data successfully exported to C:\Users\Hp/Downloads\Output_file.xlsx


In [47]:
def close_connection():
    # Close the cursor and connection
    if 'mydb' in locals() and mydb.is_connected():
        mycursor.close()
        mydb.close()
        print("MySQL connection closed.")

In [48]:
pip install pymysql openpyxl pandas sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [49]:
 close_connection()